<a href="https://colab.research.google.com/github/Ramzi7761/ventlitor/blob/main/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle (1).json to kaggle (1).json
mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d robikscube/textocr-text-extraction-from-images-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/robikscube/textocr-text-extraction-from-images-dataset
License(s): CC0-1.0
100% 6.73G/6.73G [00:44<00:00, 214MB/s]
100% 6.73G/6.73G [00:44<00:00, 164MB/s]


In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

In [ ]:
# --- 1. Load and Inspect img.parquet ---
img_df = pd.read_parquet('/content/img.parquet')
print("img.parquet DataFrame head:")
print(img_df.head())
print("\nimg.parquet DataFrame info:")
print(img_df.info())

# --- 2. Load and Inspect annot.parquet ---
annot_df = pd.read_parquet('/content/annot.parquet')
print("\nannot.parquet DataFrame head:")
print(annot_df.head())
print("\nannot.parquet DataFrame info:")
print(annot_df.info())


img.parquet DataFrame head:
                 id  width  height    set                   file_name
0  a4ea732cd3d5948a    840    1024  train  train/a4ea732cd3d5948a.jpg
1  4bf43a7b2a898044   1024     683  train  train/4bf43a7b2a898044.jpg
2  1b55b309b0f50d02   1024     683  train  train/1b55b309b0f50d02.jpg
3  00c359f294f7dcd9   1024     680  train  train/00c359f294f7dcd9.jpg
4  04b5a37f762b0f51    768    1024  train  train/04b5a37f762b0f51.jpg

img.parquet DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21778 entries, 0 to 21777
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         21778 non-null  object
 1   width      21778 non-null  int64 
 2   height     21778 non-null  int64 
 3   set        21778 non-null  object
 4   file_name  21778 non-null  object
dtypes: int64(2), object(3)
memory usage: 850.8+ KB
None

annot.parquet DataFrame head:
                   id          image_id               

In [ ]:
##3. Data Loading and Preprocessing for TextOCR ---

textocr_alphabets = u"ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.,?!:;\"'()[]{}+=-*/&%@#$<>~`|\\^_ "
max_str_len = 32
num_of_characters = len(textocr_alphabets) + 1
num_of_timestamps = 64

def textocr_label_to_num(label):
    label_num = []
    for ch in label:
        try:
            label_num.append(textocr_alphabets.find(ch))
        except:
            print(f"Character '{ch}' not found in alphabet: '{ch}'. Replacing with unknown character index.")
            label_num.append(len(textocr_alphabets)) # Index for unknown character
    return np.array(label_num)

def textocr_num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:
            break
        elif ch == len(textocr_alphabets): # Handle unknown character index
            ret += '�' # Or some placeholder character
        else:
            ret += textocr_alphabets[ch]
    return ret

def preprocess_textocr(img):
    (h, w) = img.shape
    final_img = np.ones([64, 256], dtype=np.uint8) * 255  # Ensure uint8 for image data

    if w > 256:
        img = cv2.resize(img, (256, int(h * (256/w))))
        w = 256

    if h > 64:
        img = cv2.resize(img, (int(w * (64/h)), 64))
        h = 64

    final_img[:h, :w] = img
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)


In [ ]:
# --- 4. Prepare Image Paths and Labels ---

image_dir = '/content/train_val_images/train_val_images/' # Path to your images

# Merge annotations with image info based on 'image_id'
merged_df = pd.merge(img_df, annot_df, left_on='id', right_on='image_id', how='inner')

# Filter out rows with missing 'text_string' (NaN labels)
merged_df = merged_df.dropna(subset=['utf8_string']) # Corrected column name here!
merged_df = merged_df.reset_index(drop=True)


# --- 5. Split Data into Train and Validation (Simple Random Split - Adjust as needed) ---
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(merged_df, test_size=0.2, random_state=42) # 80% train, 20% validation
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

train_size_textocr = len(train_df)
valid_size_textocr = len(valid_df)

print(f"Training set size: {train_size_textocr}")
print(f"Validation set size: {valid_size_textocr}")

Training set size: 841883
Validation set size: 210471
